In [183]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [172]:
pip install datatable

In [197]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("./drive/MyDrive/millennium_project")

import glob
import os, gc
import numpy as numpy
import pandas as pd
import scipy as sp
import datatable as dt
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
from numba import njit
from utils import *

from IPython.display import clear_output

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# TF
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [198]:
ticker_name = "JPM"

In [199]:
N_FOLD = 5
N_MINS = 5
MIN_SIZE = 600 // N_MINS

SOL_NAME = ticker_name+'-mlp'
DATA_NAME = ticker_name
mkdir(f'/content/drive/MyDrive/millennium_project/models/{SOL_NAME}/')

# Functions

In [200]:
def rmse_keras(y_true, y_pred):
    return K.sqrt(K.mean(K.square((y_true - y_pred))))

In [201]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, learning_rate):
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
    # Output layer
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('linear')(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=[rmse_keras],
    )
    return model

# Loading data

In [202]:
from datetime import datetime


In [204]:
# train
df_train = dt.fread(f'./drive/MyDrive/millennium_project/dataset/train_{DATA_NAME}_NN.csv').to_pandas()
fea_cols = fea_cols_JPM
df_train["name"] = ticker_name

# result
df_result = dt.fread(f'./drive/MyDrive/millennium_project/dataset/train.csv').to_pandas()




In [205]:
df_train = gen_row_id(df_train)
df_result = gen_row_id(df_result)

df_train = add_time_fold(df_train, N_FOLD)

In [206]:
df_train.index = df_train.row_id
df_result.index = df_result.row_id

In [208]:
df_train

,C0,Date,JPM_close,JPM_volume,SPY_close,SPY_volume,value_to_predict,corr_3,corr_6,corr_10,...,volume_10,volume_15,volume_20,volume_30,volume_60,volume_100,volume_150,name,row_id,fold
row_id,,,,,,,,,,,,,,,,,,,,,
JPM-2000-08-04 00:00:00,149,2000-08-04,52.437500,8546500,146.375000,3686600,0.844337,0.392518,0.775665,0.565606,...,52381700.0,76599400.0,98287100.0,144436500.0,346976750.0,679683950.0,1.024398e+09,JPM,JPM-2000-08-04 00:00:00,0
JPM-2000-08-07 00:00:00,150,2000-08-07,53.562500,6924000,148.125000,4159800,0.866776,0.920835,0.800864,0.681607,...,56073100.0,78453500.0,101817500.0,147698000.0,346047650.0,677746250.0,1.019302e+09,JPM,JPM-2000-08-07 00:00:00,1
JPM-2000-08-08 00:00:00,151,2000-08-08,53.500000,3458500,148.687500,3658700,0.883284,0.959711,0.783472,0.781039,...,56834500.0,78638800.0,100634100.0,147961500.0,337284300.0,665569800.0,1.011038e+09,JPM,JPM-2000-08-08 00:00:00,2
JPM-2000-08-09 00:00:00,152,2000-08-09,52.312500,6305600,147.437500,5383800,0.896887,0.872593,0.836329,0.863721,...,58486100.0,80657300.0,102408300.0,149121000.0,336886700.0,659986250.0,1.008629e+09,JPM,JPM-2000-08-09 00:00:00,3
JPM-2000-08-10 00:00:00,153,2000-08-10,51.875000,4979200,146.718750,4193400,0.907191,0.994401,0.905623,0.895037,...,59909200.0,79519100.0,102799000.0,149201800.0,333940800.0,656889300.0,1.005238e+09,JPM,JPM-2000-08-10 00:00:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JPM-2023-09-25 00:00:00,5969,2023-09-25,146.449997,7044700,432.230011,70874500,0.908433,0.545786,0.956229,0.517647,...,90504700.0,129324400.0,165868800.0,245758900.0,551979000.0,945080600.0,1.740277e+09,JPM,JPM-2023-09-25 00:00:00,0
JPM-2023-09-26 00:00:00,5970,2023-09-26,144.929993,7468400,425.880005,96168400,0.916233,0.977418,0.962914,0.776421,...,89609900.0,128938100.0,168243100.0,248183000.0,545238900.0,939618100.0,1.737749e+09,JPM,JPM-2023-09-26 00:00:00,1
JPM-2023-09-27 00:00:00,5971,2023-09-27,145.779999,6717800,426.049988,104705800,0.924372,0.842778,0.917757,0.952805,...,88001800.0,127026700.0,168623900.0,244600500.0,543935100.0,929246700.0,1.733359e+09,JPM,JPM-2023-09-27 00:00:00,2


# Evaluation

In [209]:
batch_size = 1024
hidden_units = [64, 32, 16]
dropout_rates = 0
learning_rate = 6e-3
epochs = 1000

list_seeds = [0, 11, 42, 777, 2045]

In [210]:
list_rmse = []
for i_seed, seed in enumerate(list_seeds):
    df_train = add_time_fold(df_train, N_FOLD, seed=seed)
    list_rmse += [[]]
    for i_fold in range(N_FOLD):
        gc.collect()
        df_tr = df_train.loc[df_train.fold!=i_fold]
        df_te = df_train.loc[df_train.fold==i_fold]

        X_train = df_tr[fea_cols].values
        y_train = df_tr[['value_to_predict']].values
        X_test = df_te[fea_cols].values
        y_test = df_te[['value_to_predict']].values
        idx_test = df_train.loc[df_train.fold==i_fold].index
        print(f'Fold {i_seed+1}/{len(list_seeds)} | {i_fold+1}/{N_FOLD}', X_train.shape, X_test.shape)


        # Callbacks
        ckp_path = f'./models/{SOL_NAME}/model_{i_seed}_{i_fold}.hdf5'
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, min_delta=1e-5, verbose=2)
        es = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=30, restore_best_weights=True, verbose=2)

        model = create_mlp(X_train.shape[1], 1, hidden_units, dropout_rates, learning_rate)
        history = model.fit(X_train, y_train,
            epochs=epochs,
            validation_data=(X_test, y_test),
            validation_batch_size=len(y_test),
            batch_size=batch_size,
            verbose=2,
            callbacks=[rlr, es]
        )
        # model = tf.keras.models.load_model(ckp_path, custom_objects={'mspe_loss': mspe_loss})

        y_pred = model.predict(X_test, batch_size=len(y_test))
        curr_rmse = rmse_keras(y_test, y_pred)
        list_rmse[-1] += [curr_rmse]
        model.save(ckp_path)
        # generate and save preds
        df_result.loc[idx_test, f'pred_{i_seed}'] = y_pred
        clear_output()
        print(list_rmse)

[[<tf.Tensor: shape=(), dtype=float64, numpy=0.07279401328403613>, <tf.Tensor: shape=(), dtype=float64, numpy=0.06938818847985878>, <tf.Tensor: shape=(), dtype=float64, numpy=0.06917030543052795>, <tf.Tensor: shape=(), dtype=float64, numpy=0.08354213271023285>, <tf.Tensor: shape=(), dtype=float64, numpy=0.08402775585023838>], [<tf.Tensor: shape=(), dtype=float64, numpy=0.08968880965872168>, <tf.Tensor: shape=(), dtype=float64, numpy=0.08349176915266698>, <tf.Tensor: shape=(), dtype=float64, numpy=0.08828680774261398>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0918170250714944>, <tf.Tensor: shape=(), dtype=float64, numpy=0.07602588411758907>], [<tf.Tensor: shape=(), dtype=float64, numpy=0.08631732462961546>, <tf.Tensor: shape=(), dtype=float64, numpy=0.08140310029354463>, <tf.Tensor: shape=(), dtype=float64, numpy=0.08803539232294161>, <tf.Tensor: shape=(), dtype=float64, numpy=0.08684833353874752>, <tf.Tensor: shape=(), dtype=float64, numpy=0.08973610928252819>], [<tf.Tensor: shape=

In [211]:
df_result.to_csv(f'./drive/MyDrive/millennium_project/results/{SOL_NAME}.csv', index=False)

In [212]:
for i in range(len(list_seeds)):
    print(i, rmse(df_result['value_to_predict'], df_result[f'pred_{i}']))
print('All: ', rmse(df_result['value_to_predict'], df_result[[f'pred_{i}' for i in range(len(list_seeds))]].mean(axis=1)))

0 0.07607650348808992
1 0.08604628947777172
2 0.08651309486741637
3 0.09342799773772267
4 0.08385390905459134
All:  0.07307957993178124
